In [2]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
import os
import numpy as np
import glob
from PIL import Image

In [3]:
def random_shearing(img, num, border):
    rows = img.shape[0]
    cols = img.shape[1]
    if num == 0:
        pts1 = np.float32([[5,5],[20,5],[2,20]])
        pts2 = np.float32([[10,10],[20,5],[5,25]])
    elif num == 1:
        pts1 = np.float32([[5,5],[15,5],[2,20]])
        pts2 = np.float32([[5,10],[10,10],[5,25]])
    elif num == 2:
        pts1 = np.float32([[5,5],[15,5],[5,20]])
        pts2 = np.float32([[5,10],[10,10],[5,25]])
    elif num == 3:
        pts1 = np.float32([[5,5],[10,5],[2,20]])
        pts2 = np.float32([[5,10],[10,10],[5,25]])
    elif num == 4:
        pts1 = np.float32([[5,5],[10,5],[2,20]])
        pts2 = np.float32([[5,10],[10,10],[5,30]])
    else:
        pts1 = np.float32([[5,5],[10,5],[10,20]])
        pts2 = np.float32([[5,10],[10,10],[5,30]])
    M = cv2.getAffineTransform(pts1,pts2)
    return cv2.warpAffine(img, M, (cols,rows), borderValue=border)

def random_rotation(img, degree, border):
    rows = img.shape[0]
    cols = img.shape[1]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),degree,1)
    return cv2.warpAffine(img,M,(cols,rows), borderValue=border)

In [4]:
def transform_image(img, skt, ang_range, shear_range, trans_range):
    '''
    This function transforms images to generate new images.
    The function takes in following arguments,
    1- Image
    2- ang_range: Range of angles for rotation
    3- shear_range: Range of values to apply affine transform to
    4- trans_range: Range of values to apply translations over.

    A Random uniform distribution is used to generate different parameters for transformation

    '''
    # Rotation

    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols,ch = img.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = trans_range*np.random.uniform()-trans_range/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])

    pt1 = 5+shear_range*np.random.uniform()-shear_range/2
    pt2 = 20+shear_range*np.random.uniform()-shear_range/2

    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])

    shear_M = cv2.getAffineTransform(pts1,pts2)

    # Border
    idx = 0
    border_img = tuple([int(img[idx][0][0]), int(img[idx][0][1]), int(img[idx][0][2])])
    border_skt = tuple([int(skt[0][0][0]), int(skt[0][0][1]), int(skt[0][0][2])])
    
    img = cv2.warpAffine(img,Rot_M,(cols,rows), borderValue=border_img)
    img = cv2.warpAffine(img,Trans_M,(cols,rows), borderValue=border_img)
    img = cv2.warpAffine(img,shear_M,(cols,rows), borderValue=border_img)
    
    skt = cv2.warpAffine(skt,Rot_M,(cols,rows), borderValue=border_skt)
    skt = cv2.warpAffine(skt,Trans_M,(cols,rows), borderValue=border_skt)
    skt = cv2.warpAffine(skt,shear_M,(cols,rows), borderValue=border_skt)

    return img, skt

In [5]:
sketch_dir = 'Dataset/Augmented sketch/'
photo_dir = 'Dataset/Augmented photo/'

if not os.path.exists(sketch_dir):
    os.mkdir(sketch_dir)

if not os.path.exists(photo_dir):
    os.mkdir(photo_dir)

p_filenames = sorted(glob.glob('Dataset/CUHK/Training photo/*'))
s_filenames = sorted(glob.glob('Dataset/CUHK/Training sketch/*'))

counter = 0
for i in range(len(p_filenames)):
    im = cv2.imread(p_filenames[i])
    sk = cv2.imread(s_filenames[i])

    for j in range(200):
        img, skt = transform_image(im, sk, 40, 10, 10)

        cv2.imwrite(photo_dir + str(counter) + '.jpg', img)
        cv2.imwrite(sketch_dir + str(counter) + '.jpg', skt)

        counter += 1

In [6]:
print(p_filenames)

['Dataset/CUHK/Training photo/f-005-01.jpg', 'Dataset/CUHK/Training photo/f-006-01.jpg', 'Dataset/CUHK/Training photo/f-007-01.jpg', 'Dataset/CUHK/Training photo/f-008-01.jpg', 'Dataset/CUHK/Training photo/f-009-01.jpg', 'Dataset/CUHK/Training photo/f-010-01.jpg', 'Dataset/CUHK/Training photo/f-011-01.jpg', 'Dataset/CUHK/Training photo/f-012-01.jpg', 'Dataset/CUHK/Training photo/f-013-01.jpg', 'Dataset/CUHK/Training photo/f-014-01.jpg', 'Dataset/CUHK/Training photo/f-015-01.jpg', 'Dataset/CUHK/Training photo/f-016-01.jpg', 'Dataset/CUHK/Training photo/f-017-01.jpg', 'Dataset/CUHK/Training photo/f-018-01.jpg', 'Dataset/CUHK/Training photo/f-019-01.jpg', 'Dataset/CUHK/Training photo/f-020-01.jpg', 'Dataset/CUHK/Training photo/f-021-01.jpg', 'Dataset/CUHK/Training photo/f-022-01.jpg', 'Dataset/CUHK/Training photo/f-023-01.jpg', 'Dataset/CUHK/Training photo/f-024-01.jpg', 'Dataset/CUHK/Training photo/f-025-01.jpg', 'Dataset/CUHK/Training photo/f-026-01.jpg', 'Dataset/CUHK/Training photo/f-

In [7]:
print(s_filenames)

['Dataset/CUHK/Training sketch/F2-005-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-006-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-007-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-008-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-009-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-010-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-011-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-012-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-013-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-014-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-015-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-016-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-017-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-018-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-019-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-020-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-021-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-022-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-023-01-sz1.jpg', 'Dataset/CUHK/Training sketch/F2-024-01-sz1.jpg',

In [9]:
import tensorflow as tf
tf.test.is_gpu_available()

True